# 1. 제 1 유형 

> 데이터셋: HR-Employee-Attrition.csv
* https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset
* 주어지는 데이터는 IBM 근무자에 대한 직무정보와 퇴사여부를 정리한 것이다. 

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("HR-Employee-Attrition.csv")

> (1) 데이터에서 'Attrition' 은 퇴사여부에 대한 종속변수에 해당한다. 종속변수의 값을 다음과 같이 수치로 변환해 새 칼럼으로 추가하고 범주별 레코드 수를 계산하시오. (Yes -> 1, No -> 0)


In [3]:
df["Attrition_binary"] = df.Attrition.replace({"Yes": 1, "No": 0})

In [4]:
print(df.groupby("Attrition_binary")["Attrition_binary"].count())

Attrition_binary
0    1233
1     237
Name: Attrition_binary, dtype: int64


> (2) 데이터셋의 데이터 타입별 컬럼 개수를 계산하고, 범주형 변수 중 유일한 값을 1개 만 가지고 있는 컬럼을 찾아내어 그 변수를 데이터에서 제거하시오. 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

int64 타입 27개, object(문자열) 타입 9개 

In [6]:
df.describe(include=[object]).loc["unique"]

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
Name: unique, dtype: object

Over18 칼럼이 하나의 범주만을 가지고 있다.

In [7]:
df = df.drop("Over18", axis=1)

> (3) 원래 데이터에서 숫자형인 컬럼만 추출하여 새로운 데이터 프레임을 생성하고, 각 변수간의 상관계수(피어슨상관계수)를 구하고 상관계수가 0.9 이상인 두 개의 컬럼을 찾아내어 그 변수 중 하나를 제거하시오.

In [8]:
new_df = df[df.describe(include=[np.number]).columns]

In [9]:
new_df.corr().style.background_gradient()

/usr/local/lib/python3.9/dist-packages/pandas/io/formats/style.py:3931: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.9/dist-packages/pandas/io/formats/style.py:3932: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_binary
Age,1.000000,0.010661,-0.001686,0.208034,nan,-0.010145,0.010146,0.024287,0.029820,0.509604,-0.004892,0.497855,0.028051,0.299635,0.003634,0.001904,0.053535,nan,0.037510,0.680381,-0.019621,-0.021490,0.311309,0.212901,0.216513,0.202089,-0.159205
DailyRate,0.010661,1.000000,-0.004985,-0.016806,nan,-0.050990,0.018355,0.023381,0.046135,0.002966,0.030571,0.007707,-0.032182,0.038153,0.022704,0.000473,0.007846,nan,0.042143,0.014515,0.002453,-0.037848,-0.034055,0.009932,-0.033229,-0.026363,-0.056652
DistanceFromHome,-0.001686,-0.004985,1.000000,0.021042,nan,0.032916,-0.016075,0.031131,0.008783,0.005303,-0.003669,-0.017014,0.027473,-0.029251,0.040235,0.027110,0.006557,nan,0.044872,0.004628,-0.036942,-0.026556,0.009508,0.018845,0.010029,0.014406,0.077924
Education,0.208034,-0.016806,0.021042,1.000000,nan,0.042070,-0.027128,0.016775,0.042438,0.101589,-0.011296,0.094961,-0.026084,0.126317,-0.011111,-0.024539,-0.009118,nan,0.018422,0.148280,-0.025100,0.009819,0.069114,0.060236,0.054254,0.069065,-0.031373
EmployeeCount,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
EmployeeNumber,-0.010145,-0.050990,0.032916,0.042070,nan,1.000000,0.017621,0.035179,-0.006888,-0.018519,-0.046247,-0.014829,0.012648,-0.001251,-0.012944,-0.020359,-0.069861,nan,0.062227,-0.014365,0.023603,0.010309,-0.011240,-0.008416,-0.009019,-0.009197,-0.010577
EnvironmentSatisfaction,0.010146,0.018355,-0.016075,-0.027128,nan,0.017621,1.000000,-0.049857,-0.008278,0.001212,-0.006784,-0.006259,0.037600,0.012594,-0.031701,-0.029548,0.007665,nan,0.003432,-0.002693,-0.019359,0.027627,0.001458,0.018007,0.016194,-0.004999,-0.103369
HourlyRate,0.024287,0.023381,0.031131,0.016775,nan,0.035179,-0.049857,1.000000,0.042861,-0.027853,-0.071335,-0.015794,-0.015297,0.022157,-0.009062,-0.002172,0.001330,nan,0.050263,-0.002334,-0.008548,-0.004607,-0.019582,-0.024106,-0.026716,-0.020123,-0.006846
JobInvolvement,0.029820,0.046135,0.008783,0.042438,nan,-0.006888,-0.008278,0.042861,1.000000,-0.012630,-0.021476,-0.015271,-0.016322,0.015012,-0.017205,-0.029071,0.034297,nan,0.021523,-0.005533,-0.015338,-0.014617,-0.021355,0.008717,-0.024184,0.025976,-0.130016
JobLevel,0.509604,0.002966,0.005303,0.101589,nan,-0.018519,0.001212,-0.027853,-0.012630,1.000000,-0.001944,0.950300,0.039563,0.142501,-0.034730,-0.021222,0.021642,nan,0.013984,0.782208,-0.018191,0.037818,0.534739,0.389447,0.353885,0.375281,-0.169105


In [11]:
corr = df.corr()

for col in corr.columns:
    for row in corr.index:
        if 0.9 <= corr.loc[row, col] < 1:
            print([row, col])

['MonthlyIncome', 'JobLevel']
['JobLevel', 'MonthlyIncome']


<ipython-input-11-53b2fb0a74fe>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


JobLevel 칼럼과 MonthlyIncome 칼럼럼의 피어슨 상관계수가 0.95 정도 이다.

In [12]:
df = df.drop("JobLevel", axis=1)



---



---



# 2. 제 2 유형 

> 데이터셋: Parkinsons.csv
* https://archive.ics.uci.edu/ml/datasets/parkinsons
* 환자들의 뇌를 촬영한 사진의 상태를 기록한 자료에 각 환자의 상채를 status(1: 파킨슨병 진단, 0: 파킨슨병 아님)로 추가한 테이블이다. 

> 데이터셋을 이용하려 파킨슨병을 예측하는 모델델을 로지스틱 회귀모형을 적용하여 생성하고, 이때 파킨슨병을 예측하는데 영향을 미치는 변수를 중중요한 순서대로 3개 선정하시오. <br>
이 모델에서 파킨슨병으로 진단하는 기준(threshold, 또는 cutoff)을 0.5로 했을 때와 0.8로 했을 때의 F1-score를 비교하고 해석하시오. (단, 다음 조건을 지켜서 물음에 답하시오.)

> 분석 조건
* 필요 없는 컬럼인 'name' 제거
* 로지스틱 회귀를 위해 상수항 추가
* 트레이닝셋과 테스트셋의 비율은 9:1
* 모델의 최적화 방법론으로 "bfgs" 사용
* 데이터 정규화는 min-max 스케일러 사용
* Status는 카테고리 타입으로 변환
* 모델은 로지스틱 회귀분석을 사용 

In [87]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

In [49]:
df = pd.read_csv("Parkinsons.csv")

In [15]:
df

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,phon_R01_S50_2,174.188,230.978,94.261,0.00459,0.00003,0.00263,0.00259,0.00790,0.04087,...,0.07008,0.02764,19.517,0,0.448439,0.657899,-6.538586,0.121952,2.657476,0.133050
191,phon_R01_S50_3,209.516,253.017,89.488,0.00564,0.00003,0.00331,0.00292,0.00994,0.02751,...,0.04812,0.01810,19.147,0,0.431674,0.683244,-6.195325,0.129303,2.784312,0.168895
192,phon_R01_S50_4,174.688,240.005,74.287,0.01360,0.00008,0.00624,0.00564,0.01873,0.02308,...,0.03804,0.10715,17.883,0,0.407567,0.655683,-6.787197,0.158453,2.679772,0.131728
193,phon_R01_S50_5,198.764,396.961,74.904,0.00740,0.00004,0.00370,0.00390,0.01109,0.02296,...,0.03794,0.07223,19.020,0,0.451221,0.643956,-6.744577,0.207454,2.138608,0.123306


In [50]:
df = df.drop("name", axis=1)

* 결측 없음

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MDVP:Fo(Hz)       195 non-null    float64
 1   MDVP:Fhi(Hz)      195 non-null    float64
 2   MDVP:Flo(Hz)      195 non-null    float64
 3   MDVP:Jitter(%)    195 non-null    float64
 4   MDVP:Jitter(Abs)  195 non-null    float64
 5   MDVP:RAP          195 non-null    float64
 6   MDVP:PPQ          195 non-null    float64
 7   Jitter:DDP        195 non-null    float64
 8   MDVP:Shimmer      195 non-null    float64
 9   MDVP:Shimmer(dB)  195 non-null    float64
 10  Shimmer:APQ3      195 non-null    float64
 11  Shimmer:APQ5      195 non-null    float64
 12  MDVP:APQ          195 non-null    float64
 13  Shimmer:DDA       195 non-null    float64
 14  NHR               195 non-null    float64
 15  HNR               195 non-null    float64
 16  status            195 non-null    int64  
 1

* 독립변수 종속변수 분할 및 타입 지정

In [66]:
X = df.drop("status", axis=1)
y = df["status"].astype("category")

* 9:1 분할

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=11)

* 정규화

In [76]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [77]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)
y_train = list(y_train)
y_test = list(y_test)

* 상수항 추가

In [78]:
X_train = sm.add_constant(X_train, has_constant="add")
X_test = sm.add_constant(X_test, has_constant="add")

* 모델 적합

In [79]:
model = sm.Logit(y_train, X_train)
result = model.fit(method="bfgs")

         Current function value: 0.280624
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36


/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [80]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  175
Model:                          Logit   Df Residuals:                      152
Method:                           MLE   Df Model:                           22
Date:                Wed, 26 Apr 2023   Pseudo R-squ.:                  0.4967
Time:                        01:54:27   Log-Likelihood:                -49.109
converged:                      False   LL-Null:                       -97.576
Covariance Type:            nonrobust   LLR p-value:                 2.205e-11
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -2.3559      4.862     -0.485      0.628     -11.885       7.174
MDVP:Fo(Hz)         -1.6018      3.254     -0.492      0.623      -7.980       4.776
MDVP:Fhi(Hz)        -2.0007 

p-value 가 작은 순서대로 3개 (spread2, RPDE, D2) 가 종속변수에 영향을 미치는 정도가 높다고 판단할 수 있다.

* 임계치에 따라 예측된 확률값을 1또는 0으로 분할하는 함수

In [81]:
def cut_off(y, threshold):
    Y = y.copy()
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return(Y.astype(int))

In [90]:
test_y_pred_prob = result.predict(X_test)
test_y_pred = cut_off(test_y_pred_prob, 0.8)

In [91]:
print(f1_score(y_test, test_y_pred))

0.9375


In [92]:
test_y_pred_prob = result.predict(X_test)
test_y_pred = cut_off(test_y_pred_prob, 0.5)

print(f1_score(y_test, test_y_pred))

0.8823529411764706


cut-off를 0.8로 할 때 f1-score 가 더 높다.